# Transform

Transfom represent kinds of transform matrixes, including translation, scaling, rotation and projection.

In [ ]:
from py3d import Transform
Transform()

In [ ]:
Transform(n=(2,))

Translation transformation

In [ ]:
from py3d import Vector3
Vector3([2,3,4]).as_translation()

In [ ]:
Transform.from_translation(x=1,n=(2,))

Translate a series of points

In [ ]:
points=Vector3.Rand(10)@Transform.from_translation(x=1)

Scaling transformation

In [ ]:
from py3d import Vector3
Vector3(x=1,y=2,z=(1,2,3)).as_scaling()

In [ ]:
Transform.from_scaling(x=1,y=-1,z=(1,2,3))

# Perspective projection

In [ ]:
from py3d import Transform, pi, Vector3
projection=Transform.from_perspective(pi/2, 1.2, 0, 1000)
projection

In [ ]:
from py3d import Camera,pi,Vector3,Transform
import matplotlib.pyplot as plt
import numpy
camera=Camera()
camera.transform.translation = Transform.from_translation([0,0,10])
camera.transform.rotation = Transform.from_rpy([1,1,1])
camera.projection = Transform.from_perspective(pi/3,1,0,1000)
proj = Vector3.Rand(1000).U @ camera.matrix
plt.gca().set_aspect(1)
plt.scatter(proj.x, proj.y)

In [ ]:
from py3d import Transform, Vector3
assert (Transform([
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]) == [
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]).all()
assert Transform(n=(2, 3)).shape == (2, 3, 4, 4)
assert Transform(n=(4, 5)).n == (4, 5)
assert ((Vector3(x=1).H @ Transform())[..., 0:3] == Vector3(x=1)).all()
assert (Vector3(x=1) @ Transform() == Vector3(x=1)).all()
p = Vector3([1, 2, 3])
T = Transform.from_translation([1, 2, 3])
R = Transform.from_euler("xyz", [0, 0, 1])
from numpy import allclose
assert allclose(p @ T @ R, p @ (T@R))
